#  Ejercicio para aplicar Filtro Kalman y Kalman-Azhmyakov a el análisis de series de tiempo usando el IPC de la BMV.

## Introducción

En este ejercicio realizaremos una aplicación del Filtro Kalman (KF) al análisis de series de tiempo, en específico de rendimientos, así como la aplicación de una versión del KF que denominaremos Filtro Kalman-Azhmyakov o (KAF).

El periodo que se analizará será del 1ero de enero del 2017 hasta el 30 de septiembre del mismo año.


In [2]:
import pandas_datareader.data as dr
import datetime
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

Se crean dos variables, start y end, en donde se especificara el rango de tiempo para poder hacer un query 
a la base de datos (BD), para este caso utilizaremos Yahoo! Finance.

Para poder bajar los datos de Yahoo! Finance, es necesario utilizar el ticker que utiliza la BD de Yahoo
en este caso para el IPC, el ticker es ^MXX pero por la forma en como busca los datos, haciendo uso de la URL,
es necesario usar el codigo segun aparece en ella para este caso es %5EMXX.

En otro aspecto, la libreria datetime nos ayuda a formatear 

In [3]:
start = datetime.datetime(2017,1,1)
end = datetime.datetime(2017,9,30)   

ipc = dr.DataReader("%5EMXX", 'yahoo', start, end) 

Al hacer uso de Yahoo Finance para descargar datos se obtiene un data frame con diferentes columnas Open, High, Low, Close, Adjusted Close, Volume Para terminos de analisis de datos vamos a usar la variable de Adjusted Close o Cierre ajustado
ya que integra dentro de los datos la informacion de movimientos corporativos como splits, dividendos o derechos.

## Análisis preliminar de datos y selección de series

Pandas nos permite hacer uso de los datos e y requerir datos de Yahoo! Finance o cualquier otro servicio de datos sobre acciones e índices, se crea una estructura llamada Data Frame (df o en este caso ipc), la cual esta diseñada para facilitar la manipulacion de los datos.

Se puede extraer del Data Frame (df o en este caso ipc) los datos necesarios en forma de vectores, lo cual no es recomendable para ciertas cosas por ser mas tardado y crear mas lineas.

Grafica IPC Precios de cierre ajustados la funcion iloc funciona para "partir" el data frame y los vectores en este caso nos interesa la columa 4 de Adjusted Close y los renglones del 1 al 190 (todos).

In [ ]:
ipc[['Adj Close','Volume']].plot(subplots=True, style='r', figsize=(8,5))


Podemos ver que no existe algún tipo de autocorrelacion entre los diferentes lags de la serie

In [ ]:
from pandas import Series
from matplotlib import pyplot
from pandas.tools.plotting import lag_plot

lag_plot(ipc['Return'])
pyplot.show()

En otros términos, la correlación entre $ipc_t$ y $ipc_{t-1}$ es de $0.034341$

In [ ]:
from pandas import DataFrame
from pandas import concat

df_corr = concat([ipc['Return'].shift(1), ipc['Return']], axis = 1)
df_corr.columns = ['t-1','t+1']
resultado = df_corr.corr()
print(resultado)

Por otra parte podemos asumir un supuesto de normalidad

## Filtro Kalman Clásico

## Proceso a estimar

Para poder aplicar el Filtro Kalman Clásico (KF) es necesario hacer ciertas consideraciones, basado en Welch & Bishop (2006), el proceso estocástico es lineal y se define como:

$ x_k = Ax_{k-1} + Bu_{k-1} + w_{k-1}$

con una medida de $z \in \Re^m$ como:

$z_k = Hx_k + v_k$

Con las variables aleatorias $w_k$ y $v_k$ representando el ruido del proceso y su medida respectivamente. Este ruido se asume independiente, blanco y con distribucion normal

$p(w) ~ N(0, Q)$

$p(v) ~ N(0, R)$

en la práctica, la matrix de covarianca del ruido del proceso $Q$ y la matriz de covarianza del ruido de la medida $R$ cambian en cada actualización del proceso, pero para este ejercicio se asumen constantes.


La estimacion del proceso se da con dos ecuaciones, una que actualiza y otra que mide, o de manera más concreta:

### Ecuación de Medición

$$